In [100]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math
import numpy as np



In [101]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [ ]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
pandas_interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products_10k.pkl')
#pandas_interaction_data = pandas_interaction_data.sample(1000)
pandas_interaction_data.amount = pandas_interaction_data.amount.apply(lambda x: 1 if x > 0 else 0) 
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
pandas_interaction_data.main_product_id = pandas_interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)

display(pandas_interaction_data, product_data)

,customer_id,main_product_id,amount
325791,7347782,7023883,1
133655,4395746,4233880,1
128155,4315328,4234094,1
231063,5839435,5015580,1
42949,2070488,3480813,1
...,...,...,...
187060,5196436,4823122,1
471462,9626549,3846282,1
79554,2204900,4773983,1
158028,4807671,3849864,1


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]"
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]"
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[]
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0]


In [ ]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': pandas_interaction_data['customer_id'].values,
    'main_product_id': pandas_interaction_data['main_product_id'].values,
    'amount': pandas_interaction_data['amount'].values
})
#variables 
dataset_len = pandas_interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 16
train_batch_size = 128
test_batch_size = 64
random_seed = 27

In [ ]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=random_seed, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [104]:
# Unique customer and product identifiers
unique_customer_ids = np.array(pandas_interaction_data["customer_id"].unique())
unique_product_ids = np.array(pandas_interaction_data["main_product_id"].unique())

user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(unique_customer_ids)

product_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_ids_vocabulary.adapt(unique_product_ids)
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [106]:
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
class RetailModel(tfrs.Model):
    def __init__(self, unique_product_ids_vocab, unique_customer_ids_vocab, embedding_dimension=32, metrics_batchsize=1024,
                 retrieval_loss_weight=1.0, ranking_loss_weight=1.0):
        super().__init__()
        self.retrieval_loss_weight = retrieval_loss_weight
        self.ranking_loss_weight = ranking_loss_weight
        # Embedding layers
        self.user_embedding = tf.keras.Sequential([
            unique_customer_ids_vocab,
            tf.keras.layers.Embedding(input_dim=unique_customer_ids_vocab.vocabulary_size(), output_dim=embedding_dimension)
        ])
        self.product_embedding = tf.keras.Sequential([
            unique_product_ids_vocab,
            tf.keras.layers.Embedding(input_dim=unique_product_ids_vocab.vocabulary_size(), output_dim=embedding_dimension)
        ])
        # Dense layer for transforming embeddings
        self.dense_layer = tf.keras.layers.Dense(128, activation="relu")

        # Setup retrieval task and metrics
        self.retrieval_task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model))
        )

        # Setup ranking task with pairwise hinge loss
        self.ranking_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=[tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()]
        )

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """Computes the dot product between user and product embeddings to get the interaction score."""
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # Extract embeddings
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])

        # Apply dense layers
        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_embeddings)

        # Compute scores
        positive_logits = self.dot_product_score(user_output, product_output)
        # Compute losses for both retrieval and ranking
        retrieval_loss = self.retrieval_task(user_output, product_output, compute_metrics=not training)
        ranking_loss = self.ranking_task(labels=features["amount"], predictions=positive_logits, sample_weight=features.get("weight"))

        return (self.retrieval_loss_weight * retrieval_loss +
                self.ranking_loss_weight * ranking_loss)

    def evaluate(self, validation_dataset, *args, **kwargs):
        return super().evaluate(validation_dataset, *args, **kwargs)
        # or append additional results as needed



In [107]:
model = RetailModel(product_ids_vocabulary, user_ids_vocabulary, embedding_dimension=128, retrieval_loss_weight=0.5, ranking_loss_weight=1.0)

initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), loss='categorical_crossentropy') # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [108]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3)
model.fit(cached_train, validation_freq=5, epochs=10)

Epoch 1/10
63/63 [==============================] - 3s 33ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - root_mean_squared_error: 0.9948 - categorical_accuracy: 0.0159 - auc_6: 0.0000e+00 - loss: 1368.7477 - regularization_loss: 0.0000e+00 - total_loss: 1368.7477
Epoch 2/10
63/63 [==============================] - 2s 24ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - root_mean_squared_error: 0.9912 - categorical_accuracy: 0.0000e+00 - auc_6: 0.0000e+00 - loss: 1369.0576 - regularizati

In [109]:
# Evaluate the model
k = 5
result_evaluate_train = model.evaluate(cached_train)
result_evaluate_train

63/63 [==============================] - 9s 131ms/step - factorized_top_k/top_1_categorical_accuracy: 0.9415 - factorized_top_k/top_5_categorical_accuracy: 0.9491 - factorized_top_k/top_10_categorical_accuracy: 0.9505 - factorized_top_k/top_50_categorical_accuracy: 0.9515 - factorized_top_k/top_100_categorical_accuracy: 0.9515 - root_mean_squared_error: 0.8481 - categorical_accuracy: 0.0159 - auc_6: 0.0000e+00 - loss: 969.0315 - regularization_loss: 0.0000e+00 - total_loss: 969.0315


[0.9415000081062317,
 0.9491249918937683,
 0.9505000114440918,
 0.9514999985694885,
 0.9514999985694885,
 0.8480578660964966,
 0.01587301678955555,
 0.0,
 437.1595153808594,
 0,
 437.1595153808594]

In [110]:
result_evaluat_test = model.evaluate(cached_test)
result_evaluat_test #rsme 3.3579

32/32 [==============================] - 4s 137ms/step - factorized_top_k/top_1_categorical_accuracy: 0.9955 - factorized_top_k/top_5_categorical_accuracy: 0.9965 - factorized_top_k/top_10_categorical_accuracy: 0.9970 - factorized_top_k/top_50_categorical_accuracy: 0.9980 - factorized_top_k/top_100_categorical_accuracy: 0.9980 - root_mean_squared_error: 0.8154 - categorical_accuracy: 0.0000e+00 - auc_6: 0.0000e+00 - loss: 380.5140 - regularization_loss: 0.0000e+00 - total_loss: 380.5140


[0.9955000281333923,
 0.9965000152587891,
 0.996999979019165,
 0.9980000257492065,
 0.9980000257492065,
 0.8154318928718567,
 0.0,
 0.0,
 66.9395980834961,
 0,
 66.9395980834961]

In [111]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
#print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
factorized_top_k/top_1_categorical_accuracy		0.9955000281333923
factorized_top_k/top_5_categorical_accuracy		0.9965000152587891
factorized_top_k/top_10_categorical_accuracy		0.996999979019165
factorized_top_k/top_50_categorical_accuracy		0.9980000257492065
factorized_top_k/top_100_categorical_accuracy		0.9980000257492065
root_mean_squared_error		0.8154318928718567
categorical_accuracy		0.0
auc_6		0.0


In [112]:
sample = next(iter(cached_test.take(1)))

# Extract customer_id from the sample
customer_id = sample['customer_id'].numpy()
customer_id

array([ 4275416,  5191936,  9181415,  2035007,  8891134, 10874927,
        8610367,  3736938,  1957983,  6480012,  1958913,  8908474,
        7285710,  8867556,  6373394,  2196344,  8446599,  1948734,
        6525510,  7553474,  9446582,  9965105,  2142504, 10860996,
        8121725, 10145415,  9108029, 10511849, 10170223, 10270040,
        6456133,  5530970,  3959734,  7019456,  9736062,  4100439,
        5318358,  9834638,  6798151,  2116409,  2032193,  8118166,
        7410488,  8147869,  9441353,  9971211,  8582644,  8657464,
        6799071, 10017324,  8062560,  4460873,  1992810,  7887224,
        9153335,  2006015,  8272020, 10112803,  6568213,  4336120,
        2083624, 10633283,  2201167,  8185718])

In [127]:
def get_random_customer_from_test_data():
    return next(iter(test.take(1)))['customer_id'].numpy()

def display_item_ids(item_ids):
    display(product_data[product_data.main_product_id.isin(item_ids) ])

def display_products_by_customer_id(customer_id):
    display_item_ids(pandas_interaction_data[pandas_interaction_data.customer_id ==  customer_id].main_product_id.tolist())
    

index = tfrs.layers.factorized_top_k.BruteForce(model.user_embedding, k=5)
index.index_from_dataset(
    product_ids_dataset.batch(100).map(lambda id: (id, model.product_model(id))))

def predict_user(user_id):
    print('predicting user: ', user_id)
    print('user already bought following products: ')
    display_products_by_customer_id(user_id)
    
    score, predicted_product_ids = index(np.array([user_id]))
    
    print('predicted products: ')
    display_item_ids(predicted_product_ids[0].numpy())
    print('scores: ', score[0].numpy())


user_id = get_random_customer_from_test_data()

predict_user(user_id)

predicting user:  4275416
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
512,4134223,4062862442301,Elegante Stiefelette Rauleder gelb,gelb,51.800.13,6590677,5014851,w,SK,H-W,...,3 cm - 5 cm,38.0,5.0,8.0,23.0,EVA,0.0,0.0,"[4134223, 4134220, 4222459, 4134229, 4222456, ...","[35.0, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
287,3476448,4060666816595,Elegante Stiefelette Rauleder grau,grau,52.703.39,6590678,5014851,w,PT,H-W,...,3 cm - 5 cm,43.0,9.0,16.0,23.0,TR,0.0,0.0,"[3476448, 3476441, 3476442, 3476439, 3476449, ...","[35.0, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
529,4151367,4062862442004,Elegante Stiefelette Rauleder rot,rot,51.800.15,6590677,5014851,w,SK,H-W,...,3 cm - 5 cm,38.0,5.0,8.0,23.0,EVA,0.0,0.0,"[4151367, 4234302, 4234310, 4234305, 4234311, ...","[35.0, 36.0, 37.0, 38.0, 38.5, 39.0, 40.0, 41...."
619,4822308,4062862872870,Plateau Pumps Rauleder schwarz,schwarz,61.470.17,6590677,315569,w,PT,F-S,...,8 cm - 10 cm,43.0,9.0,16.0,0.0,TPU,1.0,1.0,"[4822308, 4822301, 4822298, 4822302, 4822296, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2111,7589365,4064032406649,Sneaker low Textil multicolour,multicolour,86.989.33,4997827,315567,w,VN,F-S,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,Gummi-EVA,0.0,0.0,"[7589365, 7589356, 7589361, 7589358, 7589360, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2848,8642405,4065171453723,Elegante laarzen Suède zwart,zwart,95.629.17,6590677,315585,w,PT,H-W,...,5 cm - 8 cm,42.0,8.0,14.0,40.0,TPU,0.0,0.0,"[8642405, 10257380, 10257372]","[38.0, 42.0, 43.0]"


scores:  [0.04841844 0.04111428 0.03847473 0.03525699 0.03226946]


In [128]:
def calculate_coverage_and_scores(user_ids, total_items_count, example_user_id):
    all_scores = []
    all_predicted_product_ids = []  # Use a set to track unique recommended items

    for user_id in user_ids:
        # Call the prediction function for each user
        score, predicted_product_ids = index(np.array([user_id]))
        if user_id == example_user_id:
            print('predicting user: ', user_id)
            display_products_by_customer_id(user_id)
            print('predicted products: ')
            display_item_ids(predicted_product_ids[0].numpy())
            print(score[0].numpy(), predicted_product_ids[0].numpy())

        # Append the scores to the all_scores list
        all_scores.extend(score[0].numpy())

        # Update the set of all unique predicted product IDs
        all_predicted_product_ids.extend(predicted_product_ids[0].numpy())

    # Calculate min, max, and average scores
    min_score = np.min(all_scores)
    max_score = np.max(all_scores)
    avg_score = np.mean(all_scores)

    # Calculate item coverage and catalog coverage
    num_unique_items = len(np.unique(all_predicted_product_ids))
    catalog_coverage = num_unique_items / total_items_count

    # Print the results
    print(f"Minimum Score: {min_score}")
    print(f"Maximum Score: {max_score}")
    print(f"Average Score: {avg_score}")
    print(f"Catalog Coverage: {catalog_coverage:.2%} (Unique recommended items {num_unique_items} / Total catalog items {total_items_count})")

    return min_score, max_score, avg_score, catalog_coverage, all_predicted_product_ids

# Example usage
total_items_count = len(pandas_interaction_data.main_product_id.unique())

user_ids = pandas_interaction_data.customer_id.unique()
test_user_id = 7347782
min_score, max_score, avg_score, catalog_coverage, all_predicted_product_ids = calculate_coverage_and_scores(user_ids, total_items_count, test_user_id)

predicting user:  7347782


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
1581,7023883,4064032477885,Winterstiefelette Glattleder,,72.802.93,6590678,315580,w,PT,H-W,...,5 cm - 8 cm,44.0,9.5,17.0,26.0,TR,0.0,0.0,"[7023883, 7023906, 7016032, 6902928, 7023753, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
145,3505831,4062862951650,Spangenpump Glattleder blau,blau,56.146.66,6590678,315561,w,PT,H-W,...,3 cm - 5 cm,39.0,6.0,10.0,0.0,EVA,0.0,0.0,"[3505831, 3505828, 3505832, 3505829, 3505834, ...","[37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 41.0, 42...."
214,3591717,4062862017790,Elegante Stiefelette Glattleder blau,blau,55.501.26,6590677,5014851,w,PT,H-W,...,3 cm - 5 cm,38.0,5.0,8.0,15.0,Gummi,0.0,0.0,"[3591717, 3591726, 3591719, 3591727, 3591720, ...","[35.0, 35.5, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
610,3847235,4062862673200,Plateau Pumps Lederimitat rosa,rosa,51.260.74,6590677,315569,w,PT,H-W,...,5 cm - 8 cm,44.0,9.5,17.0,0.0,EVA,1.0,1.0,"[3847235, 4768898, 4768899, 4768896, 4768901, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
1291,6249074,4064032660553,PG1027 Boot Rauleder grün,grün,1027.10.03,3692257,3703628,m,PT,H-W,...,3 cm - 5 cm,42.0,8.0,7.0,10.0,TR,0.0,0.0,"[6249074, 6249058, 6249075, 6249079, 6249056, ...","[39.0, 40.0, 41.0, 42.0, 42.5, 43.0, 44.0, 44...."
2748,8503146,4065171582751,Sportlicher Stiefel Materialmix Leder/Lederimi...,beige,94.679.12,6590677,5014850,w,PT,H-W,...,3 cm - 5 cm,40.0,6.5,11.0,40.0,TR,0.0,0.0,"[8503146, 8503147, 8503148, 8503145, 8503151, ...","[37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40.5, 41...."


[0.04686648 0.04118669 0.03787982 0.03721044 0.03661528] [6249074 3847235 3591717 3505831 8503146]
Minimum Score: 0.023180320858955383
Maximum Score: 0.2878354787826538
Average Score: 0.04785151779651642
Catalog Coverage: 88.24% (Unique recommended items 1215 / Total catalog items 1377)


In [ ]:
display(f"min score: {min_score}, max score: {max_score}, avg_score: {avg_score}, catalog_coverage: {catalog_coverage}, min score: {min_score},")

In [116]:
# 3 epochs: min score: 0.028976473957300186, max score: 0.31238240003585815, avg_score: 0.09717307984828949, catalog_coverage: 0.8976034858387799, min score: 0.028976473957300186,
# 10 epochs: min score: 0.03581412881612778, max score: 0.41449666023254395, avg_score: 0.16217400133609772, catalog_coverage: 0.9201161946259986, min score: 0.03581412881612778,
# 100 epochs: min score: 0.04029332101345062, max score: 0.6680517196655273, avg_score: 0.21841196715831757, catalog_coverage: 0.9201161946259986, min score: 0.04029332101345062,

In [117]:
#print(all_predicted_product_ids)